## 环境配置

In [1]:
import os
import time
import numpy as np
import multiprocessing
from joblib import Parallel, delayed
# 如果提示没有 tqdm，请先运行 !pip install tqdm
from tqdm.notebook import tqdm 

# --- 1. 环境配置: 强制单线程 ---
# 在并行计算中，每个子进程内部不要再进行多线程运算(OpenMP)，
# 否则会造成 CPU 上下文切换频繁，反而变慢。
NUM_THREADS = 1 
os.environ["OMP_NUM_THREADS"]      = str(NUM_THREADS)
os.environ["MKL_NUM_THREADS"]      = str(NUM_THREADS)
os.environ["OPENBIAS_NUM_THREADS"] = str(NUM_THREADS)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(NUM_THREADS)
os.environ["NUMEXPR_NUM_THREADS"]  = str(NUM_THREADS)

# 检测系统核心数
NUM_CPU = os.cpu_count()
if NUM_CPU is None: NUM_CPU = 4
print(f'系统 CPU 核心数: {NUM_CPU}')

np.set_printoptions(suppress=True)

系统 CPU 核心数: 20


##  数据生成工具函数

In [2]:
def beta_true_gen(N, p, K, gamma):
    '''生成真实参数'''
    np.random.seed(0)  
    beta_true = np.zeros([p+1, K-1])
    for k in range(K-1):
        beta_true[:,k] = np.random.normal(size=[p+1,]) 
        beta_true[:,k] = beta_true[:,k] / np.linalg.norm(beta_true[:,k])
    # 模拟稀有类截距 (截距变得很小，使得概率很低)
    beta_true[0,:] = beta_true[0,:] + gamma * np.log(N)
    return beta_true

def X_gen(N, p=5, rho=0.5):
    '''生成特征矩阵 X'''
    mean = np.zeros(p)
    # 构造协方差矩阵
    cov = np.zeros([p,p])
    for i in range(p):
        for j in range(i,p):
            cov[i,j] = rho**(np.abs(i-j))
    cov = cov + cov.T - np.eye(p)        
    X = np.random.multivariate_normal(mean, cov, (N,)) 
    return X

def get_onehot(y, baseclass=None):
    '''One-hot 编码'''
    idx = np.squeeze(y)
    ss = len(y)
    nclass = len(np.unique(y))
    z = np.zeros([ss, nclass])
    z[np.arange(ss), idx] = 1  
    ls_class = list(np.arange(nclass))
    if baseclass is not None:
        _ = ls_class.pop(baseclass)
    return z[:, ls_class]

## 定义求解器 (Solver)

In [3]:
def mle_multilogistic_opt_ic(x, y, K0_pval, baseclass=None): 
    '''全局最大似然 GMLE (Newton-Raphson algorithm)'''
    ss, ncov = x.shape  
    K = len(np.unique(y))    
    y_onehot = get_onehot(y, baseclass) 
    dist = 1.0; niter = 0
    beta0 = np.zeros([ncov*(K-1),1])  
    alpha0 = np.log((1/K0_pval-1)/(K-1)) 
    beta0[np.arange(K-1)*ncov] = alpha0 
    
    while (dist > 1.0e-6) & (niter < 50):
        niter += 1
        beta0mat = (beta0.reshape([(K-1),ncov]).T) * 1.0
        link_mu = x @ beta0mat  
        prob = np.exp(link_mu)
        prob = prob / (1 + np.sum(prob, axis=1, keepdims=True))
        resid = y_onehot - prob
        D1 = ((x.T @ resid/ss).T.flatten()).reshape([-1,1]) 
        
        Xrep = x.reshape([1,ss,ncov]) * np.ones([K-1,1,1]) 
        XMAT = (prob.T).reshape([K-1,ss,1]) * Xrep  
        XMAT = (XMAT.transpose([1,0,2])).reshape([ss,-1])  
        D2 = -(XMAT.T @ XMAT/ss)  
        
        for i in range(K-1):    
            probtmp = (prob[:,i]) * 1.0
            weight = np.sqrt(probtmp*(1-probtmp))
            wx = weight.reshape([ss,1]) * x
            D2[i*ncov:(i+1)*ncov, i*ncov:(i+1)*ncov] = wx.T @ wx/ss   

        step = (np.linalg.inv(D2 + 1.0e-6*np.eye(ncov*(K-1)))) @ D1   
        beta1 = beta0 + step
        dist = np.mean(np.abs(beta1 - beta0))
        beta0 = beta1
    return beta0.reshape([(K-1),ncov]).T, dist, niter  

def gd_multilogistic_opt_ic(x, y, K0_pval, baseclass, alpha): 
    '''全局最大似然 GMLE (Gradient Descent algorithm)'''
    ss, ncov = x.shape  
    K = len(np.unique(y)) 
    y_onehot = get_onehot(y, baseclass) 
    dist = 1.0; niter = 0
    beta0 = np.zeros([ncov*(K-1),1]) 
    alpha0 = np.log((1/K0_pval-1)/(K-1))
    beta0[np.arange(K-1)*ncov] = alpha0
    
    while (dist > 1.0e-6) & (niter < 1000):
        niter += 1
        beta0mat = (beta0.reshape([(K-1),ncov]).T) * 1.0
        link_mu = x @ beta0mat  
        prob = np.exp(link_mu)
        prob = prob / (1 + np.sum(prob, axis=1, keepdims=True))
        resid = y_onehot - prob
        D1 = ((x.T @ resid/ss).T).reshape([-1,1])
        beta1 = beta0 + alpha * D1
        dist = np.mean(np.abs(beta1 - beta0))
        beta0 = beta1
    return beta0.reshape([(K-1),ncov]).T, dist, niter

def solve_single_pair_joblib(X_major, Y_major, X_rare, Y_rare, K0_pval):
    """
    PMLE 的并行子任务函数 (Window/Linux 通用)
    只计算 Major Class (0) 和特定的 Rare Class (k)
    """
    # 1. 拼接数据：将 Major Class 和当前的 Rare Class 拼在一起
    x_sub = np.vstack([X_major, X_rare])
    
    # 构造二分类标签: Major设为0, Rare设为1
    n_major = len(Y_major)
    n_rare = len(Y_rare)
    y_sub = np.concatenate([np.zeros(n_major), np.ones(n_rare)]).reshape(-1, 1)
    
    # 2. 二分类 Newton-Raphson 求解
    ss, ncov = x_sub.shape
    dist = 1.0
    niter = 0
    
    # 初始化参数
    beta0 = np.zeros([ncov, 1])
    alpha0 = np.log(1/K0_pval - 1)
    beta0[0] = alpha0
    
    # 迭代优化
    while (dist > 1.0e-6) & (niter < 50):
        niter += 1
        link_mu = x_sub @ beta0
        
        # Sigmoid 函数
        prob = 1.0 / (1.0 + np.exp(-link_mu))
        
        resid = y_sub - prob
        D1 = x_sub.T @ resid / ss
        
        # Hessian 矩阵
        weight = np.sqrt(prob * (1 - prob))
        wx = weight * x_sub
        # 显式删除大变量释放内存
        del weight
        D2 = wx.T @ wx / ss + 1.0e-6 * np.eye(ncov)
        del wx
        
        # 更新
        step = np.linalg.inv(D2) @ D1
        beta1 = beta0 + step
        dist = np.mean(np.abs(beta1 - beta0))
        beta0 = beta1
        
    return beta1.reshape([ncov,])

## 主仿真循环

In [4]:
# --- 参数设置 ---
N = 10**5      # 样本量
p = 10         # 特征维度
K = 11         # 类别数 (1个Major + 10个Rare)
gamma = -0.5   # 稀有度参数
nsimu = 50      # 仿真次数 (建议先跑5次看效果，正式跑可以改大)

alpha = 40     # GD 学习率
baseclass = 0
K0_pval = 0.9  

# --- 结果容器初始化 ---
t_nr = np.zeros(nsimu)
t_gd = np.zeros(nsimu)
t_pmle = np.zeros(nsimu)

# 存储每一轮的参数估计结果
beta_hat_pmle = np.zeros([nsimu, p+1, (K-1)])
beta_hat_nr   = np.zeros([nsimu, p+1, (K-1)])
beta_hat_gd   = np.zeros([nsimu, p+1, (K-1)])

# 生成真实参数 (Ground Truth)
# 注意：beta_true 是固定的，我们看谁算出来的结果离它最近
beta_true = beta_true_gen(N, p, K, gamma) 

print(f"=== 开始仿真 ===")
print(f"设置: N={N}, K={K}, p={p}, Rounds={nsimu}")
n_jobs = min(K-1, NUM_CPU)
print(f"并行后端: Joblib (loky), Workers={n_jobs}")

# 使用 tqdm 显示进度条
for b in tqdm(range(nsimu), desc="Simulation Progress"):
    
    # 1. 生成数据
    np.random.seed(b) # 确保每一轮数据是随机生成的，但不同方法处理的是同一份数据
    X = X_gen(N, p)
    X = np.hstack([np.ones([N,1]), X]) # 增加截距项

    # 计算概率并生成 Y
    prob = np.exp(X @ beta_true) 
    prob = np.hstack([np.ones([N,1]), prob])  
    prob = prob / np.sum(prob, 1).reshape([N,1])
    prob_cumsum = np.cumsum(prob, 1)   
    Y = (np.random.uniform(size=[N,1]) < prob_cumsum).astype(np.int16)
    Y = np.argmax(Y, 1) 
    
    # === 1. PMLE: Pairwise Maximum Likelihood (并行版) ===
    t_start = time.time()
    
    idx_0 = np.where(Y == 0)[0]
    X_0 = X[idx_0]
    Y_0 = Y[idx_0]
    
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(solve_single_pair_joblib)(
            X_0, Y_0,       
            X[Y == k],      
            Y[Y == k], 
            K0_pval
        ) 
        for k in range(1, K)
    )
    
    beta_hat_pmle[b] = np.array(results).T
    t_pmle[b] = time.time() - t_start
    
    # === 2. GMLE: Newton-Raphson (标准牛顿法) ===
    t_start = time.time()
    beta_hat_nr[b], _, _ = mle_multilogistic_opt_ic(X, Y, K0_pval, baseclass=baseclass)
    t_nr[b] = time.time() - t_start
    
    # === 3. GMLE: Gradient Descent (梯度下降法) ===
    t_start = time.time()
    beta_hat_gd[b], _, _ = gd_multilogistic_opt_ic(X, Y, K0_pval, baseclass, alpha)
    t_gd[b] = time.time() - t_start
    
    # 实时打印三种方法的时间
    # 如果不想刷屏，可以把 if b % 1 == 0 改成 if b % 5 == 0
    tqdm.write(f"Round {b+1}: PMLE={t_pmle[b]:.3f}s | NR={t_nr[b]:.3f}s | GD={t_gd[b]:.3f}s")

# ==========================================
# 结果评估 (计算 RMSE)
# RMSE: Root Mean Square Error (均方根误差)，越小越好
# ==========================================

def calc_rmse(beta_hat_array, beta_true):
    # beta_hat_array: (nsimu, p+1, K-1)
    # beta_true: (p+1, K-1)
    # Numpy 会自动进行广播计算 (Broadcasting)
    diff = beta_hat_array - beta_true
    mse = np.mean(diff**2)
    return np.sqrt(mse)

rmse_pmle = calc_rmse(beta_hat_pmle, beta_true)
rmse_nr   = calc_rmse(beta_hat_nr, beta_true)
rmse_gd   = calc_rmse(beta_hat_gd, beta_true)

print("\n" + "="*50)
print(f"{'Method':<15} | {'Avg Time (s)':<15} | {'RMSE (Error)':<15}")
print("-" * 50)
print(f"{'PMLE (Ours)':<15} | {np.mean(t_pmle):<15.4f} | {rmse_pmle:<15.6f}")
print(f"{'GMLE (NR)':<15} | {np.mean(t_nr):<15.4f} | {rmse_nr:<15.6f}")
print(f"{'GMLE (GD)':<15} | {np.mean(t_gd):<15.4f} | {rmse_gd:<15.6f}")
print("="*50)
print("解读：\n1. RMSE 越小说明估计越准。\n2. PMLE 的 RMSE 应该与 GMLE(NR) 几乎一致（渐近等效）。\n3. PMLE 的时间在大规模数据下应该显著少于 NR。")

=== 开始仿真 ===
设置: N=100000, K=11, p=10, Rounds=50
并行后端: Joblib (loky), Workers=10


Simulation Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Round 1: PMLE=0.889s | NR=0.975s | GD=3.162s
Round 2: PMLE=0.379s | NR=2.182s | GD=28.155s
Round 3: PMLE=0.454s | NR=1.888s | GD=12.589s
Round 4: PMLE=0.449s | NR=2.446s | GD=6.224s
Round 5: PMLE=0.435s | NR=2.598s | GD=6.212s
Round 6: PMLE=0.442s | NR=2.101s | GD=25.231s
Round 7: PMLE=0.461s | NR=1.874s | GD=25.038s
Round 8: PMLE=0.436s | NR=2.299s | GD=6.495s
Round 9: PMLE=0.331s | NR=2.196s | GD=5.595s
Round 10: PMLE=0.313s | NR=1.827s | GD=26.503s
Round 11: PMLE=0.338s | NR=2.204s | GD=25.722s
Round 12: PMLE=0.470s | NR=2.782s | GD=26.601s
Round 13: PMLE=0.423s | NR=2.593s | GD=11.990s
Round 14: PMLE=0.425s | NR=1.811s | GD=26.497s
Round 15: PMLE=0.409s | NR=2.723s | GD=5.848s
Round 16: PMLE=0.425s | NR=2.201s | GD=26.642s
Round 17: PMLE=0.457s | NR=2.479s | GD=29.164s
Round 18: PMLE=0.419s | NR=2.787s | GD=6.845s
Round 19: PMLE=0.314s | NR=2.582s | GD=6.543s
Round 20: PMLE=0.407s | NR=2.493s | GD=29.353s
Round 21: PMLE=0.328s | NR=2.399s | GD=6.955s
Round 22: PMLE=0.421s | NR=1.91

## 高维模拟

In [5]:
if __name__ == '__main__':
    # ==========================================
    # 1. 高维参数设置 (High Dimensional Settings)
    # ==========================================
    N_hd = 10**5       # 样本量: 10万
    p_hd = 500         # 特征维度: 500 (高维!)
    K_hd = 51          # 类别数: 51 (多类!)
    gamma_hd = -0.5    # 稀有度
    
    # 仿真次数
    # 高维计算比较耗时，建议先设为 2 或 5 跑通流程
    nsimu_hd = 2       
    
    alpha_hd = 40      # GD 学习率
    baseclass_hd = 0
    K0_pval_hd = 0.9

    # ==========================================
    # 2. 结果容器 (注意: 没有 NR 方法了)
    # ==========================================
    t_gd_hd = np.zeros(nsimu_hd)
    t_pmle_hd = np.zeros(nsimu_hd)

    # 参数估计结果矩阵 (维度变大)
    # Shape: [nsimu, 501, 50]
    beta_hat_pmle_hd = np.zeros([nsimu_hd, p_hd+1, (K_hd-1)])
    beta_hat_gd_hd   = np.zeros([nsimu_hd, p_hd+1, (K_hd-1)])
    
    # 生成真实参数 (Ground Truth)
    print("正在生成高维真实参数 Beta_true ...")
    beta_true_hd = beta_true_gen(N_hd, p_hd, K_hd, gamma_hd) 

    # ==========================================
    # 3. 开始仿真
    # ==========================================
    print(f"\n=== 开始高维仿真 (High-Dim) ===")
    print(f"设置: N={N_hd}, K={K_hd}, p={p_hd}, Rounds={nsimu_hd}")
    print(f"注: 由于维度过高 (Hessian矩阵 > 2.5万维), GMLE-NR 方法已被禁用。")
    
    # 并行设置
    n_jobs_hd = min(K_hd-1, NUM_CPU)
    print(f"并行后端: Joblib (loky), Workers={n_jobs_hd}")

    for b in tqdm(range(nsimu_hd), desc="HD Simulation Progress"):
        
        # --- 生成高维数据 ---
        np.random.seed(b)
        X = X_gen(N_hd, p_hd)
        X = np.hstack([np.ones([N_hd,1]), X]) # 增加截距项

        # 计算概率并生成 Y
        # 注意：这里矩阵乘法运算量较大
        prob = np.exp(X @ beta_true_hd) 
        prob = np.hstack([np.ones([N_hd,1]), prob])  
        prob = prob / np.sum(prob, 1).reshape([N_hd,1])
        prob_cumsum = np.cumsum(prob, 1)   
        Y = (np.random.uniform(size=[N_hd,1]) < prob_cumsum).astype(np.int16)
        Y = np.argmax(Y, 1) 
        
        # === 1. PMLE: Pairwise Maximum Likelihood (并行) ===
        t_start = time.time()
        
        # 提取 Major Class
        idx_0 = np.where(Y == 0)[0]
        X_0 = X[idx_0]
        Y_0 = Y[idx_0]
        
        # 启动 50 个并行任务
        # 每个任务只需要处理 p=500 的矩阵求逆，速度依然很快
        results = Parallel(n_jobs=n_jobs_hd, backend='loky')(
            delayed(solve_single_pair_joblib)(
                X_0, Y_0,       
                X[Y == k],      
                Y[Y == k], 
                K0_pval_hd
            ) 
            for k in range(1, K_hd)
        )
        
        beta_hat_pmle_hd[b] = np.array(results).T
        t_pmle_hd[b] = time.time() - t_start
        
        # === 2. GMLE: Gradient Descent (串行) ===
        # 梯度下降不需要求大矩阵的逆，所以可以运行
        # 但在高维下收敛可能很慢
        t_start = time.time()
        beta_hat_gd_hd[b], _, _ = gd_multilogistic_opt_ic(X, Y, K0_pval_hd, baseclass_hd, alpha_hd)
        t_gd_hd[b] = time.time() - t_start
        
        # 打印本轮耗时
        tqdm.write(f"Round {b+1}: PMLE={t_pmle_hd[b]:.3f}s | GD={t_gd_hd[b]:.3f}s")

    # ==========================================
    # 4. 结果评估 (计算 RMSE)
    # ==========================================
    def calc_rmse(beta_hat_array, beta_true):
        diff = beta_hat_array - beta_true
        mse = np.mean(diff**2)
        return np.sqrt(mse)

    rmse_pmle_hd = calc_rmse(beta_hat_pmle_hd, beta_true_hd)
    rmse_gd_hd   = calc_rmse(beta_hat_gd_hd, beta_true_hd)

    print("\n" + "="*60)
    print(f"High-Dimensional Result (N={N_hd}, p={p_hd}, K={K_hd})")
    print(f"{'Method':<15} | {'Avg Time (s)':<15} | {'RMSE (Error)':<15}")
    print("-" * 60)
    print(f"{'PMLE (Ours)':<15} | {np.mean(t_pmle_hd):<15.4f} | {rmse_pmle_hd:<15.6f}")
    print(f"{'GMLE (GD)':<15} | {np.mean(t_gd_hd):<15.4f} | {rmse_gd_hd:<15.6f}")
    print(f"{'GMLE (NR)':<15} | {'Failed (OOM)':<15} | {'N/A':<15}")
    print("="*60)

正在生成高维真实参数 Beta_true ...

=== 开始高维仿真 (High-Dim) ===
设置: N=100000, K=51, p=500, Rounds=2
注: 由于维度过高 (Hessian矩阵 > 2.5万维), GMLE-NR 方法已被禁用。
并行后端: Joblib (loky), Workers=20


HD Simulation Progress:   0%|          | 0/2 [00:00<?, ?it/s]

Round 1: PMLE=92.109s | GD=157.979s
Round 2: PMLE=94.634s | GD=168.552s

High-Dimensional Result (N=100000, p=500, K=51)
Method          | Avg Time (s)    | RMSE (Error)   
------------------------------------------------------------
PMLE (Ours)     | 93.3715         | 0.073824       
GMLE (GD)       | 163.2655        | 0.073095       
GMLE (NR)       | Failed (OOM)    | N/A            
